# OmniIndex
## How to use the omniindex connector for python
examples of each api call are provided in the following notebook cells

written and maintained by: [**@james stanbridge**](mailto:james@omniindex.io)

In [1]:
# mandatory cell
# expects omniindex==0.1.12 minimum
import json
from omniindex.api import OmniIndexClient # this will change in future versions to omniindex.connector
import os
import pandas as pd
import time


if you want to use another method to set your environment variables, you can do so. This is the recommomended method for use with jupyter notebooks

In [2]:
SERVER = os.environ['OI_API_TEST_SERVER'] # maps to server
API_KEY = os.environ['OI_API_TEST_API_KEY'] # maps to api_key
UNIT_NAME = os.environ['OI_API_TEST_UNIT_NAME']# maps to unit_name
USER = os.environ['OI_API_TEST_USER'] # maps to user
#
BLOCK_TYPE = "Owner"
MASTER_KEY = os.environ['OI_API_MASTER_KEY_DEMO'] # master key for demonstration

Setting up the client:
as with all the functions, you need to start with a client object = OmniIndexClient()

Parameters
- server (str) – The server to connect to. This is the node server for your blockchain
- api_key (str) – The api key also know as password for your blockchain
- unit_name (str) – The unit name for your blockchain
- user (str) – The user name for your blockchain (paired with the api key)
- block_type (str) – The block type you want to get the schematic for (typically 'Owner' or 'Global')

### get_block_schematic
it's useful to know the structure of the block you're working with. This function will return the block schematic for a given block id

In [3]:
client = OmniIndexClient(SERVER, API_KEY, UNIT_NAME, USER, BLOCK_TYPE)
block_schematic = client.get_block_schematic()
data = json.loads(block_schematic)
data

{}

generating fake-random data for the examples


In [4]:
from faker import Faker
from faker.providers import internet
""" Creates a fake info for testing purposes
    :param lcid: language codes for localisation
    :type lcid: list
    :return: 
    """

def create_info(lcid: list):
    # Create a Faker object
    fake = Faker(lcid)
    
    # Generate random data
    NAME = fake.name()
    ADDRESS = fake.address()
    INFO = fake.paragraph(nb_sentences=10, variable_nb_sentences=False)
    IPV4 = fake.ipv4()
    CREDIT = fake.credit_card_number()
    CC_EXP = fake.credit_card_expire()
    DOB = str(fake.date_of_birth())
    UUID = fake.uuid4()
    EMOJI = fake.emoji()
    JOB = fake.job()
    PHONE = fake.phone_number()
    USER_AGENT = fake.user_agent()
    
    # Create a JSON object
    data = {
        "nameEncrypt": NAME,
        "addressEncrypt": ADDRESS,
        "infoEncrypt": INFO,
        "ipv4Encrypt": IPV4,
        "creditEncrypt": CREDIT,
        "cc_exp": CC_EXP,
        "dobEncrypt": DOB,
        "uuid": UUID,
        "emoji": EMOJI,
        "job": JOB,
        "phone": PHONE,
        "user_agent": USER_AGENT
    }
    
    return data


    

In [32]:
data = create_info(['tr_TR', 'en_GB', 'ja_JP'])
data

{'nameEncrypt': 'Alaaddin Gül',
 'addressEncrypt': '719 Aylil Burg\nNew Melûl, CA 89033',
 'infoEncrypt': 'Rerum aliquid quo cumque. Ipsum soluta veritatis blanditiis repudiandae exercitationem. Minima minus cumque commodi perspiciatis itaque odit. Blanditiis ut dolorem culpa distinctio autem dignissimos. Voluptates mollitia dicta. Cumque ut sed. Illum laborum distinctio nobis recusandae repellendus. Expedita omnis aspernatur aspernatur. Tempora placeat cupiditate. Aliquam sunt ut ipsam fugiat.',
 'ipv4Encrypt': '201.218.157.233',
 'creditEncrypt': '3537647219691171',
 'cc_exp': '08/31',
 'dobEncrypt': '1964-06-11',
 'uuid': '56a62aa2-44cb-4550-b44a-4dfe5a54be80',
 'emoji': '🔎',
 'job': '映画監督',
 'phone': '(090)249 4952',
 'user_agent': 'Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_8_3; rv:1.9.2.20) Gecko/9323-01-29 13:24:07 Firefox/3.6.20'}

## minedata

Adds a new block to the blockchain.
- first cell creates a new, random data payload
- second cell adds the data to the blockchain

In [7]:
# This cell creates a payload (JSON file of records to be added)

# it is recommended practice to always add a timestamp to your data
DATE_ADDED = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
import datetime

# OmniIndex JSON parser accepts only string values
STAMP = str(datetime.datetime.now().timestamp() * 1000) 

# setup a payload to add to the block
query = create_info(['tr_TR', 'en_GB', 'ja_JP']) # added some localised data just for fun
query["dateAdded"] = DATE_ADDED
payload = json.dumps(query)
print(f"payload = {payload}")

payload = {"nameEncrypt": "Sezginba\u015f Han\u00e7er", "addressEncrypt": "58897 F\u0131rat Corners\nEvdeburgh, NM 06214", "infoEncrypt": "\u72d0\u5b66\u751f\u30af\u30eb\u30fc\u54c1\u8cea\u30b3\u30df\u30e5\u30cb\u30b1\u30fc\u30b7\u30e7\u30f3\u6301\u3064\u30b3\u30fc\u30ca\u30fc\u30ea\u30d5\u30c8\u3002\u63a2\u67fb\u5408\u8a08\u30b3\u30df\u30e5\u30cb\u30c6\u30a3\u53c2\u52a0\u3059\u308b\u30ad\u30e3\u30d3\u30cd\u30c3\u30c8\u526f\u54c1\u8cea\u30c8\u30ea\u30d3\u30e5\u30fc\u30c8\u3002\u8a3c\u8a00\u3059\u308b\u30dc\u30c8\u30eb\u30b5\u30e9\u30c0\u5968\u52b1\u3057\u307e\u3059\u8a66\u3057\u3066\u307f\u308b\u30b3\u30d4\u30fc\u30af\u30ed\u30b9\u4fdd\u8a3c\u91d1\u3002\u5927\u7d71\u9818\u90e8\u968a\u30d2\u30fc\u30eb\u3002\u72ed\u3044\u30d6\u30e9\u30b1\u30c3\u30c8\u7b26\u53f7\u8ef8\u697d\u3057\u3093\u3067\u52a9\u3051\u3066\u8a3c\u8a00\u3059\u308b\u3002\u697d\u3057\u3093\u3067\u5fc5\u8981\u611f\u8b1d\u3059\u308b\u30d0\u30ca\u30fc\u3002\u30ad\u30e3\u30d3\u30f3\u7ba1\u7406\u3059\u308b\u30ea\u30d5\u30c8\u3

In [8]:

# create a client object
client = OmniIndexClient(SERVER, API_KEY, UNIT_NAME, BLOCK_TYPE, USER)
result = client.post_minedata(MASTER_KEY, payload,)
print(f"This is the return value: {result}")

This is the return value: 


# APPENDIX
## useful tools etc

Recipes for generating usernames and passwords

In [28]:
# Generate a ten-character alphanumeric password with at least one lowercase character, at least one uppercase character, and at least three digits:
from faker import Faker


def ten_pass():
    fake = Faker()
    password = fake.password(length=10, special_chars=True, digits=True, upper_case=True, lower_case=True)
    return password

def user_name():
    fake = Faker()
    user_name = fake.user_name()
    return user_name

def pass_key():
    fake = Faker()
    api_key = fake.password(length=32, special_chars=True, digits=True, upper_case=True, lower_case=True)
    return api_key

new_user = user_name()
new_pass = ten_pass()
new_key = pass_key()

print(f"new username: {new_user}")
print(f"new password: {new_pass}")
print(f"new api key: {new_key}")


new username: pattersonveronica
new password: 1kASnvMv^X
new api key: ebX5x%a*DCTMjXOXQ(StesTVx(gi*5v!


In [11]:
print(f"Client = {client.server}, {client.api_key}, {client.unit_name}, {client.block_type}, {client.user}")
API_KEY

Client = https://node1.omniindex.xyz/node, 1kASnvMv^X, marketingdept, Owner, pattersonveronica


'1kASnvMvX'